In [ ]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
plt.rcParams['image.cmap'] = 'gray'

In [2]:

import keras

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import os
import datasets
import models
dataset="cifar10"
(x_train, y_train), (x_test, y_test), input_shape,num_classes = datasets.get_data(dataset)


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 32
data_generator=ImageDataGenerator()
train_dataset=data_generator.flow(x_train,y_train,shuffle=True,batch_size=batch_size)
test_dataset=data_generator.flow(x_test,y_test,batch_size=batch_size)

cardinality=32
width=128
model = models.residual_network(input_shape,num_classes,cardinality,width)


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=0.0015),
              metrics=['accuracy'])
print(model.summary())



x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


TypeError: residual_network() missing 1 required positional argument: 'width'

In [171]:


epochs = 15

weights_filename = 'rotation_test_model.hdf'
model.save_weights(weights_filename, overwrite=True)
if False and os.path.exists(weights_filename):
    print('Loading existing weights')
    model.load_weights(weights_filename)
else:
    model.fit_generator(train_dataset,
          steps_per_epoch=len(x_train) / batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=test_dataset)
    model.save_weights(weights_filename, overwrite=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_124 (Conv2D)             (None, 15, 15, 128)  3584        input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_125 (Conv2D)             (None, 15, 15, 32)   4128        conv2d_124[0][0]                 
__________________________________________________________________________________________________
conv2d_128 (Conv2D)             (None, 15, 15, 32)   4128        conv2d_124[0][0]                 
__________________________________________________________________________________________________
conv2d_131

Epoch 1/15
1563/1562 [==============================] - 28s 18ms/step - loss: 1.7060 - acc: 0.3800 - val_loss: 1.3798 - val_acc: 0.4976
Epoch 2/15
1563/1562 [==============================] - 27s 17ms/step - loss: 1.3052 - acc: 0.5335 - val_loss: 1.2490 - val_acc: 0.5428
Epoch 3/15
1563/1562 [==============================] - 27s 17ms/step - loss: 1.1516 - acc: 0.5925 - val_loss: 1.1233 - val_acc: 0.6056
Epoch 4/15
1563/1562 [==============================] - 27s 17ms/step - loss: 1.0832 - acc: 0.6188 - val_loss: 1.1141 - val_acc: 0.6213
Epoch 5/15
1563/1562 [==============================] - 27s 17ms/step - loss: 1.0723 - acc: 0.6283 - val_loss: 1.1984 - val_acc: 0.5871
Epoch 6/15
1563/1562 [==============================] - 27s 17ms/step - loss: 1.0647 - acc: 0.6302 - val_loss: 1.2451 - val_acc: 0.5662
Epoch 7/15
1563/1562 [==============================] - 27s 17ms/step - loss: 1.0689 - acc: 0.6285 - val_loss: 1.1028 - val_acc: 0.6265
Epoch 8/15
1563/1562 [==========================

In [173]:
rotated_epochs=25
rotated_data_generator=ImageDataGenerator(rotation_range=180)
rotated_train_dataset=rotated_data_generator.flow(x_train,y_train,shuffle=True,batch_size=batch_size)
rotated_test_dataset=rotated_data_generator.flow(x_test,y_test,batch_size=batch_size)

rotated_model = get_model(input_shape,num_classes,filters=64)

rotated_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


rotated_weights_filename = 'rotation_test_model_rotated.hdf'
if False and os.path.exists(rotated_weights_filename):
    print('Loading existing weights')
    rotated_model.load_weights(rotated_weights_filename)
else:
    rotated_model.fit_generator(rotated_train_dataset,
          steps_per_epoch=len(x_train) / batch_size,
          epochs=rotated_epochs,
          verbose=1,
          validation_data=rotated_test_dataset)
    rotated_model.save_weights(rotated_weights_filename, overwrite=True)
    


Epoch 1/25
1563/1562 [==============================] - 34s 22ms/step - loss: 1.9439 - acc: 0.2844 - val_loss: 1.7344 - val_acc: 0.3748
Epoch 2/25
1563/1562 [==============================] - 32s 21ms/step - loss: 1.7067 - acc: 0.3809 - val_loss: 1.7730 - val_acc: 0.3601
Epoch 3/25
1563/1562 [==============================] - 32s 21ms/step - loss: 1.6102 - acc: 0.4212 - val_loss: 1.5651 - val_acc: 0.4412
Epoch 4/25
1563/1562 [==============================] - 32s 21ms/step - loss: 1.5449 - acc: 0.4463 - val_loss: 1.4855 - val_acc: 0.4711
Epoch 5/25
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4897 - acc: 0.4666 - val_loss: 1.4404 - val_acc: 0.4885
Epoch 6/25
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4489 - acc: 0.4792 - val_loss: 1.4235 - val_acc: 0.4931
Epoch 7/25
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4109 - acc: 0.4966 - val_loss: 1.3888 - val_acc: 0.5004
Epoch 8/25
1563/1562 [==========================

In [174]:

frozen_layers_config=[["conv1","conv2","fc1"],
                      ["conv1","conv2","fc2"],
                      ["conv1","conv2"], 
                      #["conv2","fc1","fc2"], 
                      ["fc1","fc2"]
                      ,  ]
retrained_models=[]

for frozen_layers in frozen_layers_config:
    print("Training model with frozen layers: %s" % str(frozen_layers))
    retrained_model = get_model(input_shape,num_classes,frozen_layers=frozen_layers)

    retrained_model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    batch_size = 32
    retrained_epochs = 50

    weights_filename = 'rotation_test_model.hdf'
    retrained_model.load_weights(weights_filename)

    retrained_model.fit_generator(rotated_train_dataset,
              steps_per_epoch=len(x_train) / batch_size,
              epochs=retrained_epochs,
              verbose=1,
              validation_data=rotated_test_dataset)
    retrained_models.append(retrained_model)


Training model with frozen layers: ['conv1', 'conv2', 'fc1']
Epoch 1/50
1563/1562 [==============================] - 35s 22ms/step - loss: 1.7406 - acc: 0.3705 - val_loss: 1.6623 - val_acc: 0.3908
Epoch 2/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.6225 - acc: 0.4071 - val_loss: 1.5696 - val_acc: 0.4300
Epoch 3/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.5742 - acc: 0.4264 - val_loss: 1.5784 - val_acc: 0.4296
Epoch 4/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.5427 - acc: 0.4367 - val_loss: 1.5277 - val_acc: 0.4449
Epoch 5/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.5107 - acc: 0.4505 - val_loss: 1.5105 - val_acc: 0.4458
Epoch 6/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4886 - acc: 0.4566 - val_loss: 1.4845 - val_acc: 0.4568
Epoch 7/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4657 - acc: 0.4638 - val_loss: 1.4851 - val

1563/1562 [==============================] - 32s 21ms/step - loss: 1.4532 - acc: 0.4749 - val_loss: 1.4430 - val_acc: 0.4705
Epoch 11/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4458 - acc: 0.4781 - val_loss: 1.4480 - val_acc: 0.4836
Epoch 12/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4297 - acc: 0.4845 - val_loss: 1.4496 - val_acc: 0.4779
Epoch 13/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4186 - acc: 0.4894 - val_loss: 1.4441 - val_acc: 0.4783
Epoch 14/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4122 - acc: 0.4929 - val_loss: 1.4155 - val_acc: 0.4801
Epoch 15/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.4051 - acc: 0.4964 - val_loss: 1.4056 - val_acc: 0.5001
Epoch 16/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.3970 - acc: 0.4963 - val_loss: 1.3863 - val_acc: 0.5059
Epoch 17/50
1563/1562 [==============================

1563/1562 [==============================] - 33s 21ms/step - loss: 1.3139 - acc: 0.5302 - val_loss: 1.3373 - val_acc: 0.5234
Epoch 29/50
1563/1562 [==============================] - 33s 21ms/step - loss: 1.3044 - acc: 0.5339 - val_loss: 1.3272 - val_acc: 0.5272
Epoch 30/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.3015 - acc: 0.5339 - val_loss: 1.3027 - val_acc: 0.5351
Epoch 31/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.2983 - acc: 0.5364 - val_loss: 1.3028 - val_acc: 0.5358
Epoch 32/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.2928 - acc: 0.5375 - val_loss: 1.3076 - val_acc: 0.5308
Epoch 33/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.2869 - acc: 0.5402 - val_loss: 1.3255 - val_acc: 0.5353
Epoch 34/50
1563/1562 [==============================] - 32s 21ms/step - loss: 1.2814 - acc: 0.5423 - val_loss: 1.3067 - val_acc: 0.5423
Epoch 35/50
1563/1562 [==============================

In [156]:
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

Y_test=np.argmax(y_test, axis=1)
reports={}
scores={}
confusion_matrices={}

scores["rotated_model_test_dataset"] = rotated_model.evaluate_generator(test_dataset)
scores["model_test_dataset"] = model.evaluate_generator(test_dataset)


y_pred=model.predict(x_test)
Y_pred = np.argmax(y_pred, axis=1)
reports["model_test_dataset"]=classification_report(Y_test, Y_pred)
confusion_matrices["model_test_dataset"]=confusion_matrix(Y_test, Y_pred)

y_pred=rotated_model.predict(x_test)
Y_pred = np.argmax(y_pred, axis=1)
reports["rotated_model_test_dataset"]=classification_report(Y_test, Y_pred)
confusion_matrices["rotated_model_test_dataset"]=confusion_matrix(Y_test, Y_pred)


scores["rotated_model_rotated_test_dataset"] = rotated_model.evaluate_generator(rotated_test_dataset)
scores["model_rotated_test_dataset"] = model.evaluate_generator(rotated_test_dataset)




for i,retrained_model in enumerate(retrained_models):
    config=frozen_layers_config[i]
    key="retrained_model_test_dataset_frozen=%s" % config
    rotated_key="retrained_model_rotated_test_dataset_frozen=%s" % config
    scores[rotated_key] = retrained_model.evaluate_generator(rotated_test_dataset)
    scores[key] = retrained_model.evaluate_generator(test_dataset)
    y_pred=retrained_model.predict(x_test)
    Y_pred = np.argmax(y_pred, axis=1)
    reports[key]=classification_report(Y_test, Y_pred)
    confusion_matrices[key]=confusion_matrix(Y_test, Y_pred)

for k,v in scores.items():
    print('%s score: loss=%f, accuracy=%f' % (k,v[0],v[1]))



for k,v in reports.items():
    print('%s report: %s' % (k,v))
    
    
for k,v in confusion_matrices.items():
    print('%s conf:\n %s' % (k,v))

rotated_model_test_dataset score: loss=1.177862, accuracy=0.593800
model_test_dataset score: loss=0.785554, accuracy=0.744400
rotated_model_rotated_test_dataset score: loss=1.220749, accuracy=0.582000
model_rotated_test_dataset score: loss=2.347124, accuracy=0.343800
retrained_model_rotated_test_dataset_frozen=['conv1', 'conv2', 'fc1'] score: loss=1.614954, accuracy=0.436100
retrained_model_test_dataset_frozen=['conv1', 'conv2', 'fc1'] score: loss=1.019024, accuracy=0.676700
retrained_model_rotated_test_dataset_frozen=['conv1', 'conv2', 'fc2'] score: loss=1.381952, accuracy=0.543600
retrained_model_test_dataset_frozen=['conv1', 'conv2', 'fc2'] score: loss=1.167411, accuracy=0.619300
retrained_model_rotated_test_dataset_frozen=['conv1', 'conv2'] score: loss=1.285254, accuracy=0.568100
retrained_model_test_dataset_frozen=['conv1', 'conv2'] score: loss=1.110265, accuracy=0.628500
retrained_model_rotated_test_dataset_frozen=['conv2', 'fc1', 'fc2'] score: loss=1.704702, accuracy=0.391000
re